In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import pylab as plt
import seaborn as sns

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from lightgbm import LGBMRegressor as LGBMR

import h2o
from h2o.automl import H2OAutoML

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [3]:
salaries= pd.read_csv('../data/salaries_data.csv')

test = pd.read_csv('../data/testeo.csv')

## Limpieza

In [4]:
#Borramos columnas que no hace falta
test.drop('salary', inplace=True, axis=1)
test.drop('salary_currency', inplace=True, axis=1)

In [6]:
def diction(col):
    lista = list(set(list(salaries[col].unique()) + list(test[col].unique())))
    nums =[i for i in range(len(lista))]
    print({(lista[i]): (nums[i]) for i in range(len(lista))})
    return {(lista[i]): (nums[i]) for i in range(len(lista))}

dictio_experience_level = diction('experience_level')
dictio_employment_type = diction('employment_type')
dictio_job_title = diction('job_title')
dictio_employee_residence = diction('employee_residence')
dictio_company_location = diction('company_location')
dictio_company_size = diction('company_size')

{'SE': 0, 'EN': 1, 'EX': 2, 'MI': 3}
{'FL': 0, 'FT': 1, 'PT': 2, 'CT': 3}
{'Computer Vision Software Engineer': 0, 'Principal Data Engineer': 1, 'Computer Vision Engineer': 2, 'Marketing Data Analyst': 3, 'Director of Data Engineering': 4, 'Data Specialist': 5, 'Machine Learning Infrastructure Engineer': 6, 'Principal Data Analyst': 7, 'Principal Data Scientist': 8, 'BI Data Analyst': 9, 'Lead Data Engineer': 10, 'Lead Data Analyst': 11, 'Business Data Analyst': 12, '3D Computer Vision Researcher': 13, 'Data Science Consultant': 14, 'ETL Developer': 15, 'Director of Data Science': 16, 'Applied Data Scientist': 17, 'Data Analytics Manager': 18, 'Machine Learning Developer': 19, 'Product Data Analyst': 20, 'ML Engineer': 21, 'Machine Learning Manager': 22, 'Analytics Engineer': 23, 'Data Analytics Lead': 24, 'Finance Data Analyst': 25, 'NLP Engineer': 26, 'Head of Machine Learning': 27, 'Big Data Architect': 28, 'AI Scientist': 29, 'Head of Data': 30, 'Data Architect': 31, 'Data Analyst'

In [7]:
salaries_copy_1 = salaries.copy(deep=True)
salaries_copy_1.head()

salaries_copy_1.drop(['salary', 'salary_currency'], inplace=True,axis=1)
print(salaries_copy_1.head())

test_copy_1 = test.copy(deep=True)
print(test_copy_1.head())

   work_year experience_level employment_type        job_title  salary_in_usd  \
0       2022               SE              FT    Data Engineer         140250   
1       2022               SE              FT    Data Engineer         135000   
2       2021               MI              FT  BI Data Analyst         100000   
3       2021               MI              CT      ML Engineer         270000   
4       2021               MI              FT    Data Engineer          26005   

  employee_residence  remote_ratio company_location company_size  
0                 US           100               US            M  
1                 US           100               US            M  
2                 US           100               US            M  
3                 US           100               US            L  
4                 RO             0               US            L  
   work_year experience_level employment_type                   job_title  \
0       2020               SE     

In [8]:
test_copy_1.shape, salaries_copy_1.shape

((107, 8), (500, 9))

In [9]:
test_copy_1['y'] = -1
test_copy_1.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size,y
0,2020,SE,FT,Machine Learning Scientist,JP,0,JP,S,-1
1,2020,MI,FT,Lead Data Analyst,US,100,US,L,-1
2,2020,MI,FT,Data Analyst,US,100,US,L,-1
3,2020,MI,FT,Machine Learning Engineer,CN,0,CN,M,-1
4,2020,MI,FT,Product Data Analyst,IN,100,IN,L,-1


In [10]:
test_copy_1.shape, salaries_copy_1.shape

((107, 9), (500, 9))

In [11]:
todo = pd.concat([test_copy_1, salaries_copy_1])

todo.job_title = todo.job_title.apply(lambda x: dictio_job_title[x])
todo.employment_type = todo.employment_type.apply(lambda x: dictio_employment_type[x])
todo.employee_residence = todo.employee_residence.apply(lambda x: dictio_employee_residence[x])
todo.experience_level = todo.experience_level.apply(lambda x: dictio_experience_level[x])
todo.company_size = todo.company_size.apply(lambda x: dictio_company_size[x])
todo.company_location = todo.company_location.apply(lambda x: dictio_company_location[x])
todo.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size,y,salary_in_usd
0,2020,0,1,35,19,0,16,1,-1.0,NaN
1,2020,3,1,11,41,100,36,0,-1.0,NaN
2,2020,3,1,32,41,100,36,0,-1.0,NaN
3,2020,3,1,45,43,0,38,2,-1.0,NaN
4,2020,3,1,20,42,100,37,0,-1.0,NaN


In [12]:
ok = todo.loc[todo['y'] == -1]
ok.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size,y,salary_in_usd
0,2020,0,1,35,19,0,16,1,-1.0,NaN
1,2020,3,1,11,41,100,36,0,-1.0,NaN
2,2020,3,1,32,41,100,36,0,-1.0,NaN
3,2020,3,1,45,43,0,38,2,-1.0,NaN
4,2020,3,1,20,42,100,37,0,-1.0,NaN


In [13]:
ok.shape

(107, 10)

In [14]:
ok_s = todo.loc[todo['y'] != -1]
ok_s.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size,y,salary_in_usd
0,2022,0,1,33,41,100,36,2,NaN,140250.0
1,2022,0,1,33,41,100,36,2,NaN,135000.0
2,2021,3,1,9,41,100,36,2,NaN,100000.0
3,2021,3,3,21,41,100,36,0,NaN,270000.0
4,2021,3,1,33,14,0,36,0,NaN,26005.0


In [16]:
ok_s.shape

(500, 10)

In [17]:
ok_s.drop('y', axis=1, inplace=True)
ok.drop('salary_in_usd', axis=1, inplace=True)
ok.drop('y', axis=1, inplace=True)

## Modelo H2O

In [18]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 mins 58 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_VÃ­ctor_fugcq1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.495 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [19]:
h2train = h2o.H2OFrame(ok_s)
h2test = h2o.H2OFrame(ok)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [20]:
h2train.columns

['work_year',
 'experience_level',
 'employment_type',
 'job_title',
 'employee_residence',
 'remote_ratio',
 'company_location',
 'company_size',
 'salary_in_usd']

In [21]:
X=[c for c in h2train.columns if c!='salary_in_usd']

y='salary_in_usd'

In [22]:
automl=H2OAutoML(max_models=50,
                 seed=42,   # random_state
                 max_runtime_secs=300,
                 sort_metric='RMSE')

In [23]:
automl.train(x=X,
             y=y,
             training_frame=h2train)

AutoML progress: |█
15:59:37.659: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_1_20221203_155937_model_15


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    47                 47                          13203                  5            6            5.95745       10            22            17.8085

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 1601405335.3112514
RMSE: 40017.56283572566
MAE: 27427.1000234375
RMSLE: 0.43763545608690524
Mean Residual Deviance: 1601405335.3112514

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 2485647083.8595386
RMSE: 49856.26423890521
MAE: 33686.439698056376
RMSLE: 0.536796806473232
Mean Residual Deviance: 2485647083.8595386

Cross-Validation Metrics Summary: 
                        mean         sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  -----------  -----------  ------------  ------------  ------------  ------------  ------------
mae                     33645.8      3625.72      36577.3       30594         28933.3       35433.6       36690.9
mean_residual_deviance  2.48788e+09  8.97271e+08  3.35561e+09   1.63837e+09   1.4031e+09    3.03232e+09   3.00999e+09
mse                     2.48788e+09  8.97271e+08  3.35561e+09   1.63837e+09   1.4031e+09    3.03232e+09   3.00999e+09
r2                      0.494553     0.087521     0.414604      0.610557      0.565435      0.44603       0.43614
residual_deviance       2.48788e+09  8.97271e+08  3.35561e+09   1.63837e+09   1.4031e+09    3.03232e+09   3.00999e+09
rmse                    49158.5      9442.22      57927.6       40476.8       37458         55066.5       54863.4
rmsle                   0.532302     0.0711879    0.447608      0.630109      0.487792      0.525005      0.570995

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2022-12-03 15:59:43  2.328 sec   0                  69544.1          52000.6         4.83638e+09
    2022-12-03 15:59:43  2.332 sec   5                  58900.7          42206.2         3.46929e+09
    2022-12-03 15:59:43  2.336 sec   10                 52722.5          36680.3         2.77967e+09
    2022-12-03 15:59:43  2.340 sec   15                 48216            33017.6         2.32478e+09
    2022-12-03 15:59:43  2.343 sec   20                 45550.8          31180.7         2.07488e+09
    2022-12-03 15:59:43  2.347 sec   25                 44013.9          29966.8         1.93723e+09
    2022-12-03 15:59:43  2.351 sec   30                 42866.6          29090.5         1.83754e+09
    2022-12-03 15:59:43  2.354 sec   35                 41824.4          28463.8         1.74928e+09
    2022-12-03 15:59:43  2.358 sec   40                 40842.5          28075.9         1.66811e+09
    2022-12-03 15:59:43  2.362 sec   45                 40264.3          27570.6         1.62121e+09
    2022-12-03 15:59:43  2.363 sec   47                 40017.6          27427.1         1.60141e+09

Variable Importances: 
variable            relative_importance    scaled_importance    percentage
------------------  ---------------------  -------------------  ------------
employee_residence  2.50077e+12            1                    0.323001
company_location    1.64524e+12            0.657895             0.212501
job_title           1.59632e+12            0.638334             0.206182
experience_level    1.13127e+12            0.452371             0.146116
company_size        3.71523e+11            0.148564             0.0479862

In [24]:
print('[INFO] Leader board:')

leader_board=automl.leaderboard

leader_board.head()

[INFO] Leader board:


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_grid_1_AutoML_1_20221203_155937_model_15,49856.3,2.48565e+09,33686.4,0.536797,2.48565e+09
GBM_4_AutoML_1_20221203_155937,50018.8,2.50188e+09,33812.8,0.538568,2.50188e+09
GBM_2_AutoML_1_20221203_155937,50162,2.51623e+09,34208.7,0.548334,2.51623e+09
GBM_3_AutoML_1_20221203_155937,50178.3,2.51786e+09,34485.1,0.542477,2.51786e+09
GBM_grid_1_AutoML_1_20221203_155937_model_11,50321.2,2.53223e+09,34277.3,0.543593,2.53223e+09
GBM_grid_1_AutoML_1_20221203_155937_model_2,50431.4,2.54332e+09,34236.2,0.542066,2.54332e+09
GBM_grid_1_AutoML_1_20221203_155937_model_5,50756.9,2.57626e+09,34061,0.565723,2.57626e+09
GBM_grid_1_AutoML_1_20221203_155937_model_9,50821.5,2.58282e+09,34709.8,0.560778,2.58282e+09
GBM_grid_1_AutoML_1_20221203_155937_model_4,50861.4,2.58688e+09,34286.7,0.559155,2.58688e+09
GBM_grid_1_AutoML_1_20221203_155937_model_1,51373.4,2.63923e+09,34670.1,0.5536,2.63923e+09


In [25]:
y_pred = automl.leader.predict(h2test)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [26]:
y_pred[:10]

predict
87449
143752
123737
64931.4
71106
83133.3
125287
41264.3
63005
167199


## Exportar

In [44]:
df = y_pred.as_data_frame()

In [45]:
res = pd.read_csv('../data/muestra.csv')
res.salary_in_usd = df.predict
res.head()

,id,salary_in_usd
0,0,87449.049565
1,1,143751.753127
2,2,123737.025900
3,3,64931.437199
4,4,71105.983416


In [46]:
res.to_csv('../data/resultado_h2o.csv', index = False)